In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
! pip install scikit-optimize
# report scikit-optimize version number
import skopt
print('skopt %s' % skopt.__version__)
from skopt.space import Integer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LayerNormalization
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as K
import argparse

skopt 0.10.1


In [2]:
#data from kaggle dataset: "Prediction of music genre"
data1 = pd.read_csv("~/Downloads/music_genre.csv")
#data from kaggle data set: "Spotify Tracks Dataset"
data2 = pd.read_csv("~/Downloads/dataset.csv")

<h1>Data 2</h1>

In [3]:
#drop all the rows that are null
data2 = data2.dropna()

In [4]:
#make track_genre into label encoding
labelencoder = LabelEncoder()
data2['track_genre_num'] = labelencoder.fit_transform(data2['track_genre'])
data2

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,track_genre_num
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic,0
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic,0
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,0
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic,0
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music,113
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music,113
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music,113
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music,113


In [5]:
features = ["popularity", "duration_ms", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"]
len(features)

14

<h3>Baseline Model with SGD</h3>

In [6]:
# train 80%, val 20%, test 20%
X_train2_1b, X_test2_1b, y_train2_1b, y_test2_1b = train_test_split(data2[features], data2["track_genre_num"], test_size=0.2, random_state=1)

In [7]:
model2bs = Sequential()
model2bs.add(Dense(256, input_shape=(14,), activation="relu"))
model2bs.add(Dense(114, activation="softmax"))

sgd = SGD(0.01) #learning rate = 0.01
model2bs.compile(loss="sparse_categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"]) #use sparse for loss because label is integer and not vector
H2bs = model2bs.fit(X_train2_1b, y_train2_1b, validation_split=0.2,
	epochs=100, batch_size=64)

Epoch 1/100
1140/1140 [==============================] - 2s 2ms/step - loss: 12614315.0000 - accuracy: 0.0084 - val_loss: 4.7363 - val_accuracy: 0.0077
Epoch 2/100
1140/1140 [==============================] - 2s 2ms/step - loss: 4.7362 - accuracy: 0.0094 - val_loss: 4.7364 - val_accuracy: 0.0077
Epoch 3/100
1140/1140 [==============================] - 1s 1ms/step - loss: 4.7362 - accuracy: 0.0095 - val_loss: 4.7364 - val_accuracy: 0.0077
Epoch 4/100
1140/1140 [==============================] - 1s 1ms/step - loss: 4.7361 - accuracy: 0.0095 - val_loss: 4.7365 - val_accuracy: 0.0077
Epoch 5/100
1140/1140 [==============================] - 1s 821us/step - loss: 4.7361 - accuracy: 0.0095 - val_loss: 4.7365 - val_accuracy: 0.0077
Epoch 6/100
1140/1140 [==============================] - 1s 1ms/step - loss: 4.7361 - accuracy: 0.0095 - val_loss: 4.7366 - val_accuracy: 0.0077
Epoch 7/100
1140/1140 [==============================] - 1s 1ms/step - loss: 4.7361 - accuracy: 0.0095 - val_loss: 4.7366

In [8]:
print("Training Accuracy:", model2bs.evaluate(X_train2_1b, y_train2_1b))
print("Test Accuracy:", model2bs.evaluate(X_test2_1b, y_test2_1b))

2850/2850 [==============================] - 2s 553us/step - loss: 4.7362 - accuracy: 0.0091
Training Accuracy: [4.736191272735596, 0.009144837036728859]
713/713 [==============================] - 0s 532us/step - loss: 4.7375 - accuracy: 0.0073
Test Accuracy: [4.73750114440918, 0.007280701771378517]


<h3>Baseline Model with Adam</h3>

In [9]:
model2ba = Sequential()
model2ba.add(Dense(256, input_shape=(14,), activation="relu"))
model2ba.add(Dense(114, activation="softmax"))

model2ba.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"]) #use sparse for loss because label is integer and not vector
H2ba = model2ba.fit(X_train2_1b, y_train2_1b, validation_split=0.2,
	epochs=100, batch_size=64)

Epoch 1/100
1140/1140 [==============================] - 2s 2ms/step - loss: 6284.1514 - accuracy: 0.0091 - val_loss: 1783.6125 - val_accuracy: 0.0099
Epoch 2/100
1140/1140 [==============================] - 3s 2ms/step - loss: 1025.3003 - accuracy: 0.0089 - val_loss: 423.8664 - val_accuracy: 0.0094
Epoch 3/100
1140/1140 [==============================] - 2s 2ms/step - loss: 185.9874 - accuracy: 0.0095 - val_loss: 4.7380 - val_accuracy: 0.0082
Epoch 4/100
1140/1140 [==============================] - 1s 1ms/step - loss: 4.7377 - accuracy: 0.0087 - val_loss: 4.7376 - val_accuracy: 0.0081
Epoch 5/100
1140/1140 [==============================] - 2s 2ms/step - loss: 4.7372 - accuracy: 0.0084 - val_loss: 4.7372 - val_accuracy: 0.0081
Epoch 6/100
1140/1140 [==============================] - 2s 2ms/step - loss: 4.7367 - accuracy: 0.0083 - val_loss: 4.7370 - val_accuracy: 0.0081
Epoch 7/100
1140/1140 [==============================] - 2s 1ms/step - loss: 4.7365 - accuracy: 0.0087 - val_loss: 4.

In [25]:
print("Training Accuracy:", model2ba.evaluate(X_train2_1b, y_train2_1b))
print("Test Accuracy:", model2ba.evaluate(X_test2_1b, y_test2_1b))

2850/2850 [==============================] - 1s 374us/step - loss: 4.7362 - accuracy: 0.0091
Training Accuracy: [4.7361931800842285, 0.009144837036728859]
713/713 [==============================] - 0s 363us/step - loss: 4.7375 - accuracy: 0.0073
Test Accuracy: [4.7375054359436035, 0.007280701771378517]


In [31]:
print(classification_report(y_test2_1b, np.argmax(model2ba.predict(X_test2_1b, 1), axis =-1)))

22800/22800 [==============================] - 45s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       214
           1       0.00      0.00      0.00       202
           2       0.00      0.00      0.00       206
           3       0.00      0.00      0.00       201
           4       0.00      0.00      0.00       192
           5       0.00      0.00      0.00       199
           6       0.00      0.00      0.00       195
           7       0.00      0.00      0.00       178
           8       0.00      0.00      0.00       229
           9       0.00      0.00      0.00       206
          10       0.00      0.00      0.00       200
          11       0.00      0.00      0.00       187
          12       0.00      0.00      0.00       222
          13       0.00      0.00      0.00       203
          14       0.00      0.00      0.00       201
          15       0.00      0.00      0.00       208
          16       0.

/Users/mkobayashi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mkobayashi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mkobayashi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h3>Rescale data</h3>

In [11]:
scaler2 = preprocessing.MinMaxScaler()
scaled_feat2 = scaler2.fit_transform(data2[features])

X_train2_2, X_test2_2, y_train2_2, y_test2_2 = train_test_split(scaled_feat2, data2["track_genre_num"], test_size=0.2, random_state=1)

In [12]:
model2_2 = Sequential()
model2_2.add(Dense(256, input_shape=(14,), activation="relu"))
model2_2.add(Dense(114, activation="softmax"))

model2_2.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"]) #use sparse for loss because label is integer and not vector
H2_2 = model2_2.fit(X_train2_2, y_train2_2, validation_split=0.2,
	epochs=100, batch_size=64)

Epoch 1/100


1140/1140 [==============================] - 2s 1ms/step - loss: 3.9562 - accuracy: 0.1081 - val_loss: 3.6844 - val_accuracy: 0.1387
Epoch 2/100
1140/1140 [==============================] - 1s 800us/step - loss: 3.5908 - accuracy: 0.1576 - val_loss: 3.5416 - val_accuracy: 0.1639
Epoch 3/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.4567 - accuracy: 0.1782 - val_loss: 3.4139 - val_accuracy: 0.1805
Epoch 4/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.3440 - accuracy: 0.1931 - val_loss: 3.3264 - val_accuracy: 0.1949
Epoch 5/100
1140/1140 [==============================] - 2s 1ms/step - loss: 3.2631 - accuracy: 0.2070 - val_loss: 3.2575 - val_accuracy: 0.2004
Epoch 6/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.1996 - accuracy: 0.2153 - val_loss: 3.2065 - val_accuracy: 0.2117
Epoch 7/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.1507 - accuracy: 0.2224 - val_loss: 3.1621 - val_accuracy: 0.

In [13]:
print("Training Accuracy:", model2_2.evaluate(X_train2_2, y_train2_2))
print("Test Accuracy:", model2_2.evaluate(X_test2_2, y_test2_2))

2850/2850 [==============================] - 1s 404us/step - loss: 2.5267 - accuracy: 0.3333
Training Accuracy: [2.5267410278320312, 0.3333260118961334]
713/713 [==============================] - 0s 397us/step - loss: 2.6443 - accuracy: 0.3075
Test Accuracy: [2.6442580223083496, 0.3074561357498169]


<h3>Increasing Hidden Layers</h3>

In [14]:
model2_3 = Sequential()
model2_3.add(Dense(256, input_shape=(14,), activation="relu"))
model2_3.add(Dense(128, activation="relu"))
#LayerNormalization()
model2_3.add(Dense(114, activation="softmax"))

model2_3.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"]) #use sparse for loss because label is integer and not vector
H2_3 = model2_3.fit(X_train2_2, y_train2_2, validation_split=0.2,
	epochs=100, batch_size=64, shuffle=True)

Epoch 1/100
1140/1140 [==============================] - 2s 1ms/step - loss: 3.7763 - accuracy: 0.1259 - val_loss: 3.4675 - val_accuracy: 0.1584
Epoch 2/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.3300 - accuracy: 0.1868 - val_loss: 3.2820 - val_accuracy: 0.1867
Epoch 3/100
1140/1140 [==============================] - 1s 923us/step - loss: 3.1996 - accuracy: 0.2053 - val_loss: 3.1719 - val_accuracy: 0.2065
Epoch 4/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.1095 - accuracy: 0.2201 - val_loss: 3.1060 - val_accuracy: 0.2203
Epoch 5/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.0386 - accuracy: 0.2339 - val_loss: 3.0550 - val_accuracy: 0.2279
Epoch 6/100
1140/1140 [==============================] - 1s 982us/step - loss: 2.9806 - accuracy: 0.2433 - val_loss: 2.9962 - val_accuracy: 0.2383
Epoch 7/100
1140/1140 [==============================] - 1s 1ms/step - loss: 2.9315 - accuracy: 0.2529 - val_loss: 2.9518 - va

In [15]:
print("Training Accuracy:", model2_3.evaluate(X_train2_2, y_train2_2))
print("Test Accuracy:", model2_3.evaluate(X_test2_2, y_test2_2))

2850/2850 [==============================] - 1s 399us/step - loss: 2.3182 - accuracy: 0.3671
Training Accuracy: [2.318171739578247, 0.3670654296875]
713/713 [==============================] - 0s 532us/step - loss: 2.5974 - accuracy: 0.3152
Test Accuracy: [2.597400426864624, 0.31517544388771057]


<h3>Halving Hidden Layer Dimension</h3>

In [16]:
model2_4 = Sequential()
model2_4.add(Dense(128, input_shape=(14,), activation="relu"))
model2_4.add(Dense(64, activation="relu"))
#LayerNormalization()
model2_4.add(Dense(114, activation="softmax"))

model2_4.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"]) #use sparse for loss because label is integer and not vector
H2_4 = model2_4.fit(X_train2_2, y_train2_2, validation_split=0.2,
	epochs=100, batch_size=64, shuffle=True)

Epoch 1/100
1140/1140 [==============================] - 1s 894us/step - loss: 3.9172 - accuracy: 0.1067 - val_loss: 3.6370 - val_accuracy: 0.1419
Epoch 2/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.4919 - accuracy: 0.1671 - val_loss: 3.4008 - val_accuracy: 0.1717
Epoch 3/100
1140/1140 [==============================] - 1s 727us/step - loss: 3.3148 - accuracy: 0.1896 - val_loss: 3.3038 - val_accuracy: 0.1855
Epoch 4/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.2310 - accuracy: 0.2018 - val_loss: 3.2310 - val_accuracy: 0.1973
Epoch 5/100
1140/1140 [==============================] - 1s 957us/step - loss: 3.1737 - accuracy: 0.2113 - val_loss: 3.1764 - val_accuracy: 0.2072
Epoch 6/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.1271 - accuracy: 0.2186 - val_loss: 3.1379 - val_accuracy: 0.2113
Epoch 7/100
1140/1140 [==============================] - 1s 832us/step - loss: 3.0900 - accuracy: 0.2246 - val_loss: 3.0950 

In [17]:
print("Training Accuracy:", model2_4.evaluate(X_train2_2, y_train2_2))
print("Test Accuracy:", model2_4.evaluate(X_test2_2, y_test2_2))

2850/2850 [==============================] - 1s 351us/step - loss: 2.5468 - accuracy: 0.3224
Training Accuracy: [2.546779155731201, 0.3223719596862793]
713/713 [==============================] - 0s 364us/step - loss: 2.6311 - accuracy: 0.3046
Test Accuracy: [2.631105422973633, 0.30460527539253235]


<h3>Add Hidden Layer</h3>

In [28]:
model2_5 = Sequential()
model2_5.add(Dense(128, input_shape=(14,), activation="relu"))
model2_5.add(Dense(64, activation="relu"))
model2_5.add(Dense(32, activation="relu"))
model2_5.add(Dense(114, activation="softmax"))

model2_5.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"]) #use sparse for loss because label is integer and not vector
H2_5 = model2_5.fit(X_train2_2, y_train2_2, validation_split=0.2,
	epochs=100, batch_size=64, shuffle=True)

Epoch 1/100
1140/1140 [==============================] - 1s 875us/step - loss: 3.9685 - accuracy: 0.0963 - val_loss: 3.6872 - val_accuracy: 0.1374
Epoch 2/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.5202 - accuracy: 0.1609 - val_loss: 3.4204 - val_accuracy: 0.1715
Epoch 3/100
1140/1140 [==============================] - 1s 934us/step - loss: 3.3185 - accuracy: 0.1893 - val_loss: 3.2866 - val_accuracy: 0.1887
Epoch 4/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.2185 - accuracy: 0.2040 - val_loss: 3.2235 - val_accuracy: 0.2018
Epoch 5/100
1140/1140 [==============================] - 1s 903us/step - loss: 3.1545 - accuracy: 0.2137 - val_loss: 3.1579 - val_accuracy: 0.2086
Epoch 6/100
1140/1140 [==============================] - 1s 1ms/step - loss: 3.1071 - accuracy: 0.2207 - val_loss: 3.1037 - val_accuracy: 0.2178
Epoch 7/100
1140/1140 [==============================] - 1s 878us/step - loss: 3.0676 - accuracy: 0.2271 - val_loss: 3.0876 

In [29]:
print("Training Accuracy:", model2_5.evaluate(X_train2_2, y_train2_2))
print("Test Accuracy:", model2_5.evaluate(X_test2_2, y_test2_2))

2850/2850 [==============================] - 1s 445us/step - loss: 2.5699 - accuracy: 0.3186
Training Accuracy: [2.5699307918548584, 0.31864383816719055]
713/713 [==============================] - 0s 426us/step - loss: 2.6577 - accuracy: 0.3029
Test Accuracy: [2.657667636871338, 0.3028508722782135]


In [32]:
print(classification_report(y_test2_2, np.argmax(model2_5.predict(X_test2_2, 1), axis =-1)))

22800/22800 [==============================] - 45s 2ms/step
              precision    recall  f1-score   support

           0       0.15      0.21      0.18       214
           1       0.44      0.24      0.31       202
           2       0.02      0.01      0.01       206
           3       0.14      0.03      0.06       201
           4       0.31      0.21      0.25       192
           5       0.26      0.09      0.13       199
           6       0.50      0.48      0.49       195
           7       0.38      0.53      0.44       178
           8       0.17      0.09      0.12       229
           9       0.15      0.16      0.15       206
          10       0.49      0.44      0.46       200
          11       0.06      0.02      0.03       187
          12       0.24      0.23      0.23       222
          13       0.51      0.46      0.49       203
          14       0.33      0.41      0.36       201
          15       0.23      0.40      0.29       208
          16       0.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3da032f1-e5ab-4726-ac09-eb3d9c053730' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>